In [20]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from numpy import radians, sin, cos, arccos 
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import mannwhitneyu
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.cluster import DBSCAN
from scipy.stats import zscore
import h3
from scipy.stats import entropy
import hashlib
from math import radians, cos, sin, asin, sqrt 
import folium 

In [2]:
df = pd.read_csv('../data/raw_data/train.csv')

In [ ]:
df['amount_log'] = np.log1p(df['amount'])
df['population_log'] = np.log1p(df['population_city'])
df['amount_per_person'] = df['amount'] / (df['population_city']) 
df['amount_per_person'].replace([np.inf, -np.inf], 0, inplace=True) 
df['transaction_time'] = pd.to_datetime(df['transaction_time'])
df['hour'] = df['transaction_time'].dt.hour
df['weekday'] = df['transaction_time'].dt.weekday
df['is_night'] = df['hour'].apply(lambda h: 1 if h < 6 or h > 22 else 0)
df['user_id'] = (
    df['name_1'].astype(str) + '_' + df['name_2'] + '_' + df['street']
).map(lambda x: hashlib.md5(x.encode()).hexdigest())

df = df.sort_values('transaction_time')

In [4]:
split_point = int(len(df) * 0.8)
train = df.iloc[:split_point].copy()
test = df.iloc[split_point:].copy()

transactions_ok = train[train["target"] == 0].copy()

In [5]:
grouped = transactions_ok.groupby(["user_id", "cat_id"])["amount"].agg(
    mean_amount="mean",
    median_amount="median",
    std_amount="std",
    p95_amount=lambda x: x.quantile(0.95),
    count="count"
).reset_index()

mean_pivot = grouped.pivot(index="user_id", columns="cat_id", values="mean_amount").add_prefix("mean_cat_")
median_pivot = grouped.pivot(index="user_id", columns="cat_id", values="median_amount").add_prefix("median_cat_")
std_pivot = grouped.pivot(index="user_id", columns="cat_id", values="std_amount").add_prefix("std_cat_")
p95_pivot = grouped.pivot(index="user_id", columns="cat_id", values="p95_amount").add_prefix("p95_cat_")
count_pivot = grouped.pivot(index="user_id", columns="cat_id", values="count").add_prefix("count_cat_")

In [6]:
profile_df = pd.concat(
    [mean_pivot, median_pivot, std_pivot, p95_pivot, count_pivot],
    axis=1
).reset_index()
profile_df = profile_df.fillna(0)

profile_pivot = grouped.rename(columns={ 
    'mean_amount': 'user_cat_mean_amount',
    'median_amount': 'user_cat_median_amount',
    'std_amount': 'user_cat_std_amount',
    'p95_amount': 'user_cat_p95_amount',
    'count': 'user_cat_txn_count_in_profile'
}).copy()

In [ ]:
transactions_ok_with_profile = transactions_ok.merge(
    profile_pivot[['user_id', 'cat_id', 'user_cat_mean_amount', 'user_cat_median_amount', 'user_cat_std_amount', 'user_cat_p95_amount']],
    on=['user_id', 'cat_id'],
    how='left'
)

transactions_ok_with_profile['z_score'] = (
    (transactions_ok_with_profile['amount'] - transactions_ok_with_profile['user_cat_mean_amount']) / transactions_ok_with_profile['user_cat_std_amount']
)
transactions_ok_with_profile['delta_from_median'] = (
    transactions_ok_with_profile['amount'] - transactions_ok_with_profile['user_cat_median_amount']
)
transactions_ok_with_profile['above_p95'] = (
    transactions_ok_with_profile['amount'] > transactions_ok_with_profile['user_cat_p95_amount']
).astype(int)

transactions_ok_with_profile['z_score'].replace([np.inf, -np.inf], np.nan, inplace=True)
transactions_ok_with_profile['z_score'].fillna(0, inplace=True)
transactions_ok_with_profile['delta_from_median'].fillna(0, inplace=True)

In [8]:
deviation_profile = (
    transactions_ok_with_profile 
    .groupby('user_id')
    .agg(
        mean_z_score=('z_score', 'mean'),
        std_z_score=('z_score', 'std'),
        p95_z_score=('z_score', lambda x: np.percentile(x.dropna(), 95) if not x.dropna().empty else 0), # Добавлена обработка пустых серий
        frac_above_p95=('above_p95', 'mean'),
        mean_delta_from_median=('delta_from_median', 'mean'),
        std_delta_from_median=('delta_from_median', 'std'),
    )
    .reset_index()
    .fillna(0)
)

In [9]:
profile_df = (
    profile_df
    .merge(deviation_profile, on='user_id', how='left')
    .fillna(0)
)

In [10]:
weekday_pivot = transactions_ok.pivot_table(
    index='user_id',
    columns='weekday',
    values='transaction_time',
    aggfunc='count'
).fillna(0)
weekday_pivot.columns = [f'weekday_count_{col}' for col in weekday_pivot.columns]
weekday_pivot = weekday_pivot.reset_index()

hour_pivot = transactions_ok.pivot_table(
    index='user_id',
    columns='hour',
    values='transaction_time',
    aggfunc='count'
).fillna(0)
hour_pivot.columns = [f'hour_count_{col}' for col in hour_pivot.columns]
hour_pivot = hour_pivot.reset_index()

profile_df = profile_df.merge(weekday_pivot, on='user_id', how='left').fillna(0) 
profile_df = profile_df.merge(hour_pivot, on='user_id', how='left').fillna(0)    

txn_counts = transactions_ok.groupby('user_id').size().rename('txn_count').reset_index()
profile_df = profile_df.merge(txn_counts, on='user_id', how='left').fillna(0) 

weekday_cols = [col for col in profile_df.columns if col.startswith('weekday_count_')]
hour_cols = [col for col in profile_df.columns if col.startswith('hour_count_')]


for col in weekday_cols + hour_cols:
    profile_df[col] = np.where(profile_df['txn_count'] > 0, profile_df[col] / profile_df['txn_count'], 0)

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

user_coords = transactions_ok.groupby('user_id').agg(
    mean_lat=('lat', 'mean'),
    mean_lon=('lon', 'mean')
).reset_index()

transactions_ok_with_user_coords = transactions_ok.merge(user_coords, on='user_id', how='left')

transactions_ok_with_user_coords['geo_distance'] = transactions_ok_with_user_coords.apply(
    lambda row: haversine(row['lat'], row['lon'], row['mean_lat'], row['mean_lon'])
    if pd.notnull(row['mean_lat']) and pd.notnull(row['mean_lon']) else np.nan, 
    axis=1
)
transactions_ok_with_user_coords['geo_distance'].fillna(0, inplace=True) 

geo_profile = transactions_ok_with_user_coords.groupby('user_id').agg(
    mean_geo_distance=('geo_distance', 'mean'),
    max_geo_distance=('geo_distance', 'max'),
    std_geo_distance=('geo_distance', 'std'),
    geo_unique_locations=('lat', lambda x: pd.Series(list(zip(x, transactions_ok_with_user_coords.loc[x.index, 'lon']))).nunique()) 
).reset_index().fillna(0) 

In [12]:
profile_df = profile_df.merge(geo_profile, on='user_id', how='left').fillna(0)

In [13]:
user_coords_for_merge = user_coords.rename(columns={
    'mean_lat': 'user_profile_mean_lat',
    'mean_lon': 'user_profile_mean_lon'
})

In [14]:
def create_transaction_deviation_features(df, profile_pivot_ref, user_coords_ref):
    df_enriched = df.copy()

    df_enriched = df_enriched.merge(
        profile_pivot_ref[['user_id', 'cat_id', 'user_cat_mean_amount', 'user_cat_median_amount', 'user_cat_std_amount', 'user_cat_p95_amount']],
        on=['user_id', 'cat_id'],
        how='left'
    )

    df_enriched['txn_z_score_vs_user_cat'] = \
        (df_enriched['amount'] - df_enriched['user_cat_mean_amount']) / df_enriched['user_cat_std_amount']
    df_enriched['txn_delta_vs_user_cat_median'] = \
        df_enriched['amount'] - df_enriched['user_cat_median_amount']
    df_enriched['txn_above_user_cat_p95'] = \
        (df_enriched['amount'] > df_enriched['user_cat_p95_amount']).astype(int)
    df_enriched['txn_z_score_vs_user_cat'].replace([np.inf, -np.inf], np.nan, inplace=True)

    fill_zero_cols = [
        'user_cat_mean_amount', 'user_cat_median_amount', 'user_cat_std_amount', 'user_cat_p95_amount',
        'txn_z_score_vs_user_cat', 'txn_delta_vs_user_cat_median'
    ]
    for col in fill_zero_cols:
        df_enriched[col].fillna(0, inplace=True)

    df_enriched['txn_above_user_cat_p95'] = \
        (df_enriched['amount'] > df_enriched['user_cat_p95_amount']).astype(int)


    df_enriched = df_enriched.merge(
        user_coords_for_merge[['user_id', 'user_profile_mean_lat', 'user_profile_mean_lon']],
        on='user_id',
        how='left'
    )

    df_enriched['txn_distance_from_user_home_base'] = df_enriched.apply(
        lambda row: haversine(row['lat'], row['lon'], row['user_profile_mean_lat'], row['user_profile_mean_lon'])
        if pd.notnull(row['user_profile_mean_lat']) and pd.notnull(row['user_profile_mean_lon']) and pd.notnull(row['lat']) and pd.notnull(row['lon'])
        else np.nan,
        axis=1
    )
   
    df_enriched['txn_distance_from_user_home_base'].fillna(
        df_enriched['txn_distance_from_user_home_base'].median(), inplace=True 
    )
    df_enriched['txn_distance_from_user_home_base'].fillna(0, inplace=True) 
    
    return df_enriched

In [ ]:
train = create_transaction_deviation_features(train, profile_pivot, user_coords_for_merge)
test = create_transaction_deviation_features(test, profile_pivot, user_coords_for_merge)

In [16]:
train = train.merge(profile_df.add_prefix('user_profile_'), left_on='user_id', right_on='user_profile_user_id', how='left')
test = test.merge(profile_df.add_prefix('user_profile_'), left_on='user_id', right_on='user_profile_user_id', how='left')

if 'user_profile_user_id' in train.columns:
    train.drop(columns=['user_profile_user_id'], inplace=True)
if 'user_profile_user_id' in test.columns:
    test.drop(columns=['user_profile_user_id'], inplace=True)

In [17]:
profile_cols_in_final_df = [col for col in train.columns if col.startswith('user_profile_')]
train[profile_cols_in_final_df] = train[profile_cols_in_final_df].fillna(0)
test[profile_cols_in_final_df] = test[profile_cols_in_final_df].fillna(0)

In [ ]:
train.to_csv('train_clean.csv', index=False)

In [ ]:
test.to_csv('test_clean.csv', index=False)